In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

__file__ = '/content/drive/MyDrive/aiffelthon/data'

Mounted at /content/drive


## Env
- 경로 이동 제한 사항 반영: del apply_action() 수정
- def grid_box(self):  # 그리드 박스 초기화 용도로 정의
- 그리드값 변경
  - target_gridval (목표물)  = 10, 9, 8, ...
  - curloc_gridval (현 위치) = 1
  - default_gridval (기본)   = 0
  - rack_gridval (선반)      = -1
  - obs_gridval (장애물)     = -10
- 종료 조건
  - **그리드월드 밖**으로 나가는 경우 **종료하지 않음**
  - **장애물**에 부딪히는 경우 **종료**

In [2]:
from string import ascii_uppercase
import numpy as np
import pandas as pd
import os
import random
from datetime import datetime
import pytz
import matplotlib.pyplot as plt

###################################
# 보상 Reward
move_reward = -1  # 0.1
obs_reward = -10  # 0.1
goal_reward = 10  # 10
###################################
# 그리드값
# target_gridval = 10, 9, 8... # 목표물: 10, 9, 8, ...
curloc_gridval  = 1            # 현 위치: 1
default_gridval = 0            # 기본: 0
rack_gridval    = -1           # 선반: -1
obs_gridval     = -10          # 장애물: -10
###################################
# train / test 모드 지정
train_mode = True
###################################
print('reward:' , move_reward, obs_reward, goal_reward)

#__file__ = '/home/ogangza/heung_path_finding/path-finding-rl/data'

local_path = os.path.abspath(os.path.join(os.path.dirname(__file__)))

class Simulator:
    def __init__(self):
        # Load train or test data
        if train_mode:  # 훈련 데이타 읽기
            self.files = pd.read_csv(os.path.join(local_path, "data/factory_order_train.csv"))
            print('data/factory_order_train.csv used')
        else:           # 테스트 데이터 읽기
            self.files = pd.read_csv(os.path.join(local_path, "data/factory_order_test.csv"))
            print('data/factory_order_test.csv used')
        ##########################################################################################
        self.height = 10  # 그리드 높이
        self.width = 9    #  그리드 너비
        self.inds = list(ascii_uppercase)[:17]  # A ~ Q alphabet list

    def set_box(self):  # 선반 위치, 목적지 그리드값 설정. 목적지 리스트 생성
        box_data = pd.read_csv(os.path.join(local_path, "data/box.csv"))
        for box in box_data.itertuples(index = True, name ='Pandas'):  # 선반 위치: 그리드값 = rack_gridval
            self.grid[getattr(box, "row")][getattr(box, "col")] = rack_gridval
        order_item = list(set(self.inds) & set(self.items))
        order_csv = box_data[box_data['item'].isin(order_item)]
        for i, order_box in enumerate(order_csv.itertuples(index = True, name ='Pandas')):  # 목적지: 그리드값 = target_gridval
            self.grid[getattr(order_box, "row")][getattr(order_box, "col")] = 10 - i
            self.local_target.append([getattr(order_box, "row"), getattr(order_box, "col")])  # 목적지 리스트 생성: local_target
        # self.local_target.sort() # 불필요. 인풋 데이터 A, B, C순 정렬되어 있음... 정렬 필요시 코드 바꾸어야 함
        self.local_target.append([9,4]) # 목적지 리스트에 최종 목적지(9,4) 추가
        self.local_target_original = self.local_target.copy()  # gif 생성을 위해 추가.  에피소드의 경로 저장
        self.target_length = len(self.local_target_original)  # 목적지 그리드값 10, 9, 8, ... 지정 위해 추가
        self.grid[9,4] = 10 + 1 - self.target_length  # 최종 목적지 [9,4] 그리드값 지정 위해 추가

    def set_obstacle(self):  # 장애물 위치 그리드값 설정 = obs_gridval
        obstacles_data = pd.read_csv(os.path.join(local_path, "data/obstacles.csv"))
        for obstacle in obstacles_data.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(obstacle, "row")][getattr(obstacle, "col")] = obs_gridval

    ########################################################################################################
    '''
    writer 신휘정
    step에서 아이템을 먹으면 self.items의 첫번째 요소를 pop해줍니다.
    그리고 새로운 item에 대해서 grid_box를 그려줍니다.
    '''
    ########################################################################################################
    def grid_box(self):  # 그리드 박스 초기화 용도로 정의: 선반, 목적지, 장애물 그리드값 초기화
        box_data = pd.read_csv(os.path.join(local_path, "data/box.csv"))
        for box in box_data.itertuples(index = True, name ='Pandas'):  # 선반 위치: 그리드값 = rack_gridval
            self.grid[getattr(box, "row")][getattr(box, "col")] = rack_gridval
        # order_item = list(set(self.inds) & set(self.items))
        if len(self.items) > 0:
            order_item = list(self.items)
            order_csv = box_data[box_data['item'].isin(order_item)]
            for i, order_box in enumerate(order_csv.itertuples(index = True, name ='Pandas')):  # 목적지: 그리드값 = target_gridval
                self.grid[getattr(order_box, "row")][getattr(order_box, "col")] = 10 - i
            self.grid[9,4] = 10 + 1 - self.target_length  # 최종 목적지 [9,4] 그리드값 지정 위해 추가
        else : 
            self.grid[9,4] = 10
        self.set_obstacle()  # 장애물 위치 그리드값 설정 = obs_gridval
        
    def reset(self, epi):  # 에피소드 시작시 12개 값 초기화
        self.epi = epi                                            #1. 에피소드 번호 받기
        self.items = eval(list(self.files.iloc[self.epi])[0])           #2. 에피소드의 목적지 받기: [ 'H', 'L', 'M']
        #####################################################################################
        # coin = random.randint(0,16)
        # self.items = [chr(coin+65)]  # A~Q 까지 1개 랜덤으로 받아서 items에 1개 넣어주기, 반복 횟수는 main의 epochs 조절하기
        ####################################################################################
        self.cumulative_reward = 0                                #3. 누적 보상값 = 0
        self.terminal_location = None                             #4. 최초 목적지
        self.local_target = []                                    #5. 목적지 리스트 초기화
        self.actions = []                                         #6. 지나온 경로 리스트 초기화
        self.grid = np.zeros((self.height, self.width), dtype="float16")  #7. 그리드월드 전체 그리드값(default_gridval) 초기화
        self.set_box()                                            #8. 선반 위치, 목적지 그리드값 설정. 목적지 리스트 생성
        self.set_obstacle()                                       #9. 장애물 그리드값 설정
        ######################
        # print('최초 그리드맵:')  # 그리드맵 확인
        # print(self.grid.reshape(10,9))
        ######################
        self.curloc = [9, 4]                                      #10. 현재 위치를 출발점으로 세팅
        self.grid[int(self.curloc[0])][int(self.curloc[1])] = curloc_gridval  #11. 현재 위치(출발점) 그리드값 세팅
        self.done = False                                         #12. 경로 찾기 종료 여부 = False
        self.goal_ob_reward = False                      # (추가) #13. 최종 목적지 도착 여부 = False
        ######################
        # print('현 위치 추가:')   # 그리드맵 확인
        # print(self.grid.reshape(10,9))
        ######################
        self.move_track = [[9,4]]                                  #14. 이동하는 경로를 저장하는 변수, 에피소드 시작시 에이전트가 [9,4]에 있으므로 [[9,4]]로 초기화
        self.isReset =  False                                      #15. 처음으로 terminal_location == 9,4가 되면, moveTrack을 초기화해주기 위해 필요한 변수 isFinal -> isReset 로 변경(처음으로 불러졌냐 안불러졌냐)
        self.prior_distance = 9999                                 #16. 이전 time step의 거리와 현재 거리를 비교하기 위해서 필요한 변수
        ######################
        self.wasGoal = False                                       # 17. 이전 step이 Goal이었는지 확인하는 변수
                                                                   # 이전 step이 Goal이었으면 먹고 나가야 하는 곳이 move_track이 곂치기 때문에 -보상받음. 이를 방지해기위한 FLAG
        return self.grid

    def apply_action(self, action, cur_x, cur_y):  # action에 따른 새 좌표값 반환
        new_x = cur_x
        new_y = cur_y
        if action == 0:          # up
            #new_x = cur_x - 1
            new_x = self.move_up(cur_x, cur_y, new_x, new_y)
        elif action == 1:        # down
            #new_x = cur_x + 1
            new_x = self.move_down(cur_x, cur_y, new_x, new_y)
        elif action == 2:        # left
            #new_y = cur_y - 1
            new_y = self.move_left(cur_x, cur_y, new_x, new_y)
        else:                    # right
            #new_y = cur_y + 1
            new_y = self.move_right(cur_x, cur_y, new_x, new_y)
        return int(new_x), int(new_y)
    
    # >>> 현재 위치에서 이동이 불가한 위치 추가 (시작) <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    def move_up(self, cur_x, cur_y, new_x, new_y):  # action == 0:
        if cur_x in [6,5,4,3,2] and cur_y in [0,8]:
            pass
        else:
            new_x = cur_x - 1
        return new_x
    def move_down(self, cur_x, cur_y, new_x, new_y): # action == 1:
        if cur_x in [1,2,3,4,5] and cur_y in [0,8]:
            pass
        else:
            new_x = cur_x + 1
        return new_x
    def move_left(self, cur_x, cur_y, new_x, new_y): # left elif action == 2:
        if cur_y in [1,2,3,4,5,6,7,8] and cur_x == 0:
            pass
        else:
            new_y = cur_y - 1
        return new_y
    def move_right(self, cur_x, cur_y, new_x, new_y): # right else: action == 3:
        if cur_y in [0,1,2,3,4,5,6,7] and cur_x == 0:
            pass
        else:
            new_y = cur_y + 1
        return new_y
    # >>> 추가 (끝) <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

    def get_reward(self, new_x, new_y, out_of_boundary):  # action에 의해 이동시 얻는 보상
        if any(out_of_boundary):                                        # 바깥으로 나가는 경우
            reward = obs_reward
        else:
            if self.grid[new_x][new_y] == obs_gridval :     # 장애물에 부딪히는 경우
                reward = obs_reward
                self.grid[new_x][new_y] = curloc_gridval   # 현 위치(장애물)에서 종료되었음을 표시. 원래는 장애물이었음                
            elif self.grid[new_x][new_y] == rack_gridval:   # 선반에 부딪히는 경우
                reward = obs_reward
                new_x = self.curloc[0]  # 선반인 경우 못들어가고, 현재 위치로 유지
                new_y = self.curloc[1]  # 선반인 경우 못들어가고, 현재 위치로 유지
            elif new_x == self.terminal_location[0] and new_y == self.terminal_location[1]:  # 현재 목적지에 도달한 경우
                reward = goal_reward
                self.prior_distance = 9999                  # 목적지에 갈 때 prior_distance = 1이라, 나갈 때 무조건 prior_distance가 작아서 negative_reward를 받기때문에 여기서 못받도록 수정해줌.
            # 그냥 움직이는 경우 
            else: 
                cur_distance = self.get_distance()
                if self.prior_distance > cur_distance :                         # 이전의 거리 > 현재의 거리 = 현재 아이템에 더 가깝다 => + 보상얻기
                    reward = self.get_positive_distance_reward(cur_distance)
                else:                                                           # 이전의 거리 < 현재의 거리 = 현재 아이템에서 멀어졌다 => - 보상얻기
                    reward = self.get_negative_distance_reward(cur_distance)
                self.prior_distance = cur_distance      

                if [new_x, new_y] not in self.move_track:                           # 이동한 곳이 처음온 곳이면 move_track에 추가
                    self.move_track.append([new_x, new_y])
                else:                                                           # 이동한 곳이 이미 왔던 곳이면 기존 보상에서 -1
                    reward = reward -1
        return reward

    def step(self, action):  # action에 따른 이동 실행
        self.terminal_location = self.local_target[0]           # 목적지 리스트의 첫 번째 요소를 목적지로 설정
        cur_x, cur_y = self.curloc                              # 현재 위치 기억
        new_x, new_y = self.apply_action(action, cur_x, cur_y)  # 다음 위치 받기
        self.actions.append((cur_x, cur_y))                     # 현재 위치(지나온 위치)를 경로 리스트에 추가
        self.goal_ob_reward = False                             # 최종 목적지에 도착한 경우에 True. self.reset()에서 초기화
        out_of_boundary = [new_x < 0, new_x >= self.height, new_y < 0, new_y >= self.width]  # 그리드월드 밖으로 나갔는가? OB = True

        if any(out_of_boundary):  #1. 바깥으로 나가는 경우, 빈 선반에 부딪치는 경우 종료하지 않음
            #print('OB')
            new_x = cur_x  # 종료 처리할 경우 코멘트 처리
            new_y = cur_y  # 종료 처리할 경우 코멘트 처리
            ######################################################종료 처리할 경우 실행
            #self.done = True                             # while loop 종료
            #self.grid_box()                              # 선반, 목적지, 장애물 그리드값 초기화
            #self.actions.append((new_x,new_y))           # 경로 리스트에 최종 위치 추가. (주의) 좌표값이 그리드월드를 벗어나는 값 발생!
            ######################################################
        else:
            if self.grid[new_x][new_y] == obs_gridval:  #2. 장애물에 부딪히는 경우 종료
                # print('장애물')
                # new_x = cur_x  # 종료 처리할 경우 코멘트 처리
                # new_y = cur_y  # 종료 처리할 경우 코멘트 처리
                ###################################################종료 처리할 경우 실행
                self.done = True                           # while loop 종료
                self.grid[cur_x][cur_y] = default_gridval  # 현 위치의 그리드값을 기본값으로 초기화
                self.grid_box()                            # 선반, 목적지, 장애물 그리드값 초기화
                self.actions.append((new_x,new_y))         # 경로 리스트에 최종 위치 추가
                ###################################################
            elif self.grid[new_x][new_y] == rack_gridval:
                pass
            elif new_x == self.terminal_location[0] and new_y == self.terminal_location[1]:  #3. 현재 목적지에 도착한 경우 다음 목적지 설정
                #print('목적지 도착')
                if [new_x, new_y] == [9, 4]:            #3-1. 최종 목적지에 도착한 경우 종료
                    self.done = True                       # while loop 종료
                    self.goal_ob_reward = True             # True = 1 (OB, 장애물 종료 처리 할 경우 self.done=True가 많아지므로 이때 사용)
                    self.actions.append((new_x,new_y))     # 경로 리스트에 최종 위치 추가
                ########################################################################################################
                '''
                writer 신휘정
                기존 : 아이템을 먹어도 에피소드 동안 목적지, 선반 숫자는 유지됨.
                현재 : 아이템을 먹으면 남은 아이템에 대해서 숫자가 새로 칠해짐
                '''
                if len(self.items) > 0:
                    self.items.remove(self.items[0])            # 아이템을 하나먹었으니, 아이템의 첫번째 값을 삭제해줌
                    self.wasGoal = True
                    self.move_track.pop()                       # 아이템을 들어가고 나오는 위치는 동일(주황색동그라미) => 이미 왔던경로라고 - 리워드 못받게 하기위해 마지막 요소 pop()
                ########################################################################################################                    

                self.local_target.remove(self.local_target[0])  # 다음 목적지 설정. 최종 목적지에 도착한 경우에는 마지막 요소인 [9,4]를  제거
                self.grid[cur_x][cur_y] = default_gridval  # 현 위치의 그리드값을 기본값으로 초기화
                self.grid_box()                            # 선반, 목적지, 장애물 그리드값 초기화
                self.grid[new_x][new_y] = curloc_gridval   # 현 위치(목적지)에서 종료되었음을 표시. 원래는 목적지이었음
                self.curloc = [new_x, new_y]               # 새로 도착한 위치를 현재 위치로 변경
            
            else:                                       #4. 그냥 움직이는 경우
                #print('이동')
                ########################################################################################################
                '''
                writer 신휘정
                기존 : 아이템을 먹어도 에피소드 동안 목적지, 선반 숫자는 유지됨.
                현재 : 아이템을 먹으면 해당 선반은 빈선반으로 바뀜
                '''
                if self.wasGoal:
                    self.grid[cur_x][cur_y] = rack_gridval     # 현 위치의 그리드값을 선반값으로 초기화
                    self.wasGoal = False
                else:
                    self.grid[cur_x][cur_y] = default_gridval  # 현 위치의 그리드값을 기본값으로 초기화
                ########################################################################################################
                self.grid_box()                            # 선반, 목적지, 장애물 그리드값 초기화
                self.grid[new_x][new_y] = curloc_gridval   # 현 위치에서 종료되었음을 표시
                self.curloc = [new_x,new_y]

        ############################### 보상받는 처리 시작 ###############################
        # 마지막 위치에서 위로 올라가는 행동을 한다면 - 보상받음 태양
        if len(self.local_target) == 1:                             # local_target이 remove되고 나면 길이가 1이되는 경우 = [9,4]만 있는 경우
            if not self.isReset:                                    # 처음으로 terminal_location == [9,4]가 되었다면,,
                self.move_track = []                                # 아이템을 찾으러 돌아다니는 경로 초기화
                self.isReset = True                                 # move_track이 초기화 되었으니 isReset = True
            reward = self.get_reward(new_x, new_y, out_of_boundary)
            if action == 0:
                reward = obs_reward                                # terminal_location == [9,4]로 갈 때 위로 올라가는거 금지
        else : 
            reward = self.get_reward(new_x, new_y, out_of_boundary)
        ############################### 보상받는 처리 끝 ###############################   

        self.cumulative_reward += reward
        return self.grid, reward, self.cumulative_reward, self.done

    # >>> distance를 입력으로 받아 + 보상을 계산하는 메소드 (시작) <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 
    def get_positive_distance_reward(self, distance):
        positive_reward = (20 - distance) / 200     # 0.095 ~ 0.001
        return positive_reward
    # >>> 추가 (끝) <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

    # >>> distance를 입력으로 받아 - 보상을 계산하는 메소드 (시작) <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    def get_negative_distance_reward(self, distance):
        negative_reward = -(distance / 200)
        return negative_reward
    # >>> 추가 (끝) <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<    

    # >>> 현재 위치에서 타겟까지의 거리를 구하는 메소드 (시작) <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    def get_distance(self): 
        cur_x, cur_y, target_x, target_y = self.get_current_state()        
        distance = (abs(cur_x-target_x) + abs(cur_y-target_y))
        # print(cur_x, cur_y, target_x, target_y)
        # print('distance : ', distance)
        return distance
    # >>> 추가 (끝) <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<    

    # >>> 현재 상황에 대한 정보를 구하는 메소드 (시작) <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    def get_current_state(self):
        # if len(self.local_target) != 1:
        #     state = (self.curloc[0], self.curloc[1], self.local_target[0][0], self.local_target[0][1])
        # else :
        state = (self.curloc[0], self.curloc[1], self.terminal_location[0], self.terminal_location[1])
        return np.array(state) #(cur_x, cur_y, tar_x, tar_y)
    # >>> 추가 (끝) <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<        


reward: -1 -10 10


## Agent
- def test_action(self, obs): 추가

In [3]:
import collections
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class ReplayBuffer():  # 리플레이 버퍼: 경험 저장소
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition):  # 리플레이 버퍼(메모리)를 경험(transition)으로 채우기
        self.buffer.append(transition)
    
    def sample(self, n):  # memory.sample(batch_size)로 사용
        mini_batch = random.sample(self.buffer, n)  # 미니배치 샘플링
        '''
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        for transition in mini_batch:  # 경험의 각 요소들을 각각의 미니배치로 구성
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)
        '''
        state1_batch = torch.cat([s1 for (s1,a,r,s2,d) in mini_batch])
        action_batch = torch.Tensor([[a] for (s1,a,r,s2,d) in mini_batch])      # a를 tensor에 넣을 때는 [리스트]에 꼭 씌워주세요
        reward_batch = torch.Tensor([[r] for (s1,a,r,s2,d) in mini_batch])      # reward를 tensor에 넣을 때는 [리스트]에 꼭 씌워주세요
        state2_batch = torch.cat([s2 for (s1,a,r,s2,d) in mini_batch])
        done_batch = torch.Tensor([[d] for (s1,a,r,s2,d) in mini_batch])        # done를 tensor에 넣을 때는 [리스트]에 꼭 씌워주세요

        return state1_batch, action_batch, reward_batch, state2_batch, done_batch
    
    def size(self):  # 메모리 크기
        return len(self.buffer)

class Qnet(nn.Module):
    '''##### Linear 모델 #####
    def __init__(self):
        super(Qnet, self).__init__()

        in_size = 90  # input 크기
        L1 = 250
        L2 = 150
        out_size  = 4

        self.fc1 = nn.Linear(in_size, L1)
        self.fc2 = nn.Linear(L1, L2)
        self.fc3 = nn.Linear(L2, out_size)

    def forward(self, x):
        x = x.to(device)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    '''
    ##### Convolution 모델 #####
    def __init__(self):
        super(Qnet, self).__init__()

        self.conv1 = nn.Conv2d(4, 32, 3, 1) #### 1이었던 부분을 4로 변경
        self.conv2 = nn.Conv2d(32, 64, 3, 1) #### in_채널, out_channel, kernel_size, stride
        self.conv3 = nn.Conv2d(64, 64, 3, 1)
        self.fc1 = nn.Linear(768, 512)
        self.fc2 = nn.Linear(512, 4)

    def forward(self, x):
        x = x.to(device)               # (N, 4, 10, 9)
        x = F.relu(self.conv1(x))      # (N, 32, 8, 7)
        x = F.relu(self.conv2(x))      # (N, 64, 6, 5)
        x = F.relu(self.conv3(x))
        x = torch.flatten(x, 1)        # (N, 64, 30)
        x = F.relu(self.fc1(x))        # (N, 1920) -> (N, 128)
        x = self.fc2(x)                # (N, 128)  -> (N, 4)
        return x

    def sample_action(self, obs, epsilon):  # Qnet()을 실행해서 모든 action에 대한 Qval을 구한 후 action 선택
        out = self.forward(obs)  # out = Q_value, obs = state = 그리드맵
        coin = random.random()  # e-greedy로 action 선택
        if coin < epsilon:
            return random.randint(0,3)
        else : 
            return out.argmax().item()
    
    def test_action(self, obs):  # Qnet()을 실행해서 모든 action에 대한 Qvalue을 구한 후 action 선택
        out = self.forward(obs)  # out = Q_value, obs = state = 그리드맵
        return out.argmax().item()

##################################################### DOUBLE _ DQN ######################################################       
def train(q, q_target, memory, optimizer):  # 메모리에 경험이 미니배치 크기 이상 쌓이면 미니배치 훈련 시작
    s,a,r,s_prime,done_mask = memory.sample(batch_size)  # 메모리에서 미니배치를 뽑기: batch_size=1600
    
    q_out = q(s)  # Qnet의 Qvalue
    q_a = q_out.gather(dim=1,index=a.long().to(device)) # Qnet의 Qvalue 변환

    with torch.no_grad():
        q_prime_idx = q(s_prime).max(1)[1]                                      # output : tensor (8)
        max_q_primes = q_target(s_prime)                                        # output : tensor (8, 4)
        max_q_prime = max_q_primes.gather(1, q_prime_idx.unsqueeze(1))          # output : tensor (8, 1)

    Y = r.to(device) + gamma * max_q_prime * done_mask.to(device)               # output : tensor (8, 1)

    loss = F.smooth_l1_loss(q_a, Y.detach())
    #loss = nn.MSELoss(X.to(torch.float32), Y.detach().to(torch.float32))  # 손실함수 변경. target.detach()로 변경
    optimizer.zero_grad()  # gradient값 초기화
    loss.backward()  # 자동미분
    optimizer.step()  # gradient 업데이트
    return loss.item()
##################################################### DOUBLE _ DQN ######################################################       

########################################################## DQN ##########################################################
# def train(q, q_target, memory, optimizer):  # 메모리에 경험이 미니배치 크기 이상 쌓이면 미니배치 훈련 시작
#     '''
#     for i in range(10):
#         s, a, r, cr, s_prime, done_mask, gr = memory.sample(batch_size)
#         q_out = q(s)
#         q_a = q_out.gather(1,a.to(device))
        
#         max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
#         target = r.to(device) + gamma * max_q_prime * done_mask.to(device)
        
#         loss = F.smooth_l1_loss(q_a, target)
        
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#     '''
    # s,a,r,s_prime,done_mask = memory.sample(batch_size)  # 메모리에서 미니배치를 뽑기: batch_size=1600
    # Q1 = q(s)  # Qnet의 Qvalue
    # X = Q1.gather(dim=1,index=a.long().unsqueeze(dim=1).to(device)).squeeze() # Qnet의 Qvalue 변환
    # with torch.no_grad():
    #     Q2 = q_target(s_prime)
    # Y = r.to(device) + gamma * done_mask.to(device) * torch.max(Q2,dim=1)[0]
    # loss = F.smooth_l1_loss(X, Y.detach())
    # #loss = nn.MSELoss(X.to(torch.float32), Y.detach().to(torch.float32))  # 손실함수 변경. target.detach()로 변경
    # optimizer.zero_grad()  # gradient값 초기화
    # loss.backward()  # 자동미분
    # optimizer.step()  # gradient 업데이트
    # return loss.item()
########################################################## DQN ##########################################################

## Main: DQN...
  - GPU를 사용할 경우 구현
  - 학습한 모델 불러올 수 있는 코드 추가
    - '#' 제거하고 실행시키면 됨

In [ ]:
#def main(): # DDQN... Path 학습 모델

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # GPU를 사용할 경우 ???
print('tensor in', device)  # GPU 사용 확인
# PATH = '/content/drive/MyDrive/aiffelthon/data/model_conv_colab.pt'
PATH = __file__ + '/eps.pt' ## PATH 경로 쉽게 수정
##############################################
tz = pytz.timezone('Asia/Seoul')
cur_time = datetime.now(tz)
start_time = cur_time.strftime("%H:%M:%S")

### 중요 Hyperparameters #####################
buffer_limit   = 100000     #1. 100000
batch_size     = 200        #2. 200
learning_rate  = 0.00025    #3. 0.00025
sync_freq      = 500        #4. 500: q 네트워크 파라미터를 q_target 네트워크에 복사하는 주기
train_start    = 50000      #5. 50000: (추가) 값이 작을 경우 경험이 아래 쪽에 편중됨
gamma = 0.99                #6. 0.99
# epochs = len(env.files)   #7. 훈련용 데이터 갯수
epochs         = 50000      #7. 한 군데 갔다 오는 경우에 사용. 반복 횟수는 각자 지정
max_moves      = 200        #8. 100
print_interval = 1000       #9. 100
# epsilon = 0.3             #10. annealing 대신 고정
##############################################
env = Simulator()  # 데이터셋 읽기. 그리드월드 크기 지정
q = Qnet()
q.to(device)
optimizer = optim.Adam(q.parameters(), lr=learning_rate)

train_model_load = False  # 학습한 모델을 불러와서 계속 학습시키고자 하는 경우 True로 바꾼다!!
if train_model_load:
    ###############################################################
    ## 학습한 모델 불러오기
    ###############################################################
    checkpoint = torch.load(PATH)
    q.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    q.train()
###############################################################
q_target = Qnet()
q_target.to(device)
q_target.load_state_dict(q.state_dict())
memory = ReplayBuffer()
### Hyperparameters #####################
losses = []
cum_rewards =[]
moves = []
j = 0
goal_ob_reward_count = 0 #9. epoch를 반복하면서 finish 카운팅
loss = 0.0
##############################################
# __file__ = '/content/drive/MyDrive/aiffelthon/data' 
### 액션 저장하는 txt파일 만들기 #######################
f = open(__file__ + '/eps_gif' + '.txt', 'w')
########################################################

for n_epi in range(epochs):
    reset_epi = n_epi % 39999
    epsilon = max(0.05, 1.00 - 0.1*(n_epi/2000)) # Linear annealing from 100% to 10%
    gridmap = env.reset(reset_epi)  # 에피소드 시작시 12개 값 초기화
    gridmap[9,4] = curloc_gridval                                   #0. 현재 gridmap에 [9,4]를 1로 표시
        # self.epi = epi                                            #1. 에피소드 번호 받기
        # self.items = list(self.files.iloc[self.epi])[0]           #2. 에피소드의 목적지 받기: [ 'H', 'L', 'M']
        # self.cumulative_reward = 0                                #3. 누적 보상값 = 0
        # self.terminal_location = None                             #4. 최초 목적지
        # self.local_target = []                                    #5. 목적지 리스트 초기화
        # self.actions = []                                         #6. 지나온 경로 리스트 초기화
        # self.grid = np.ones((self.height, self.width), dtype="float16")  #7. 그리드월드 전체 그리드값(1)로 초기화
        # self.set_box()                                            #8. 선반 위치(0), 목적지(2) 그리드값 설정.  목적지 리스트 생성
        # self.set_obstacle()                                       #9. 장애물 그리드값(0) 설정
        # self.curloc = [9, 4]                                      #10. 현재 위치를 출발점으로 세팅
        # self.grid[int(self.curloc[0])][int(self.curloc[1])] = -1  #11. 현재 위치(출발점) 그리드값(-1) 세팅
        # self.done = False                                         #12. 경로 찾기 종료 여부 = False
        # return self.grid

    # 초기 input은 (시작화면*4장)으로 들어감
    #####################################################################################
    history1 = np.stack((gridmap,gridmap,gridmap,gridmap), axis = 0) # 초기 state로 히스토리 초기화 (4,10,9)
    history1 = np.reshape([history1], (1,4,10,9)) # (배치, 채널, 가로, 세로) 로 reshape
    history1 = torch.from_numpy(history1).float() # 텐서로 변환   
    #####################################################################################

    done = False
    mov = 0

    while (not done):  # 최종 목적지 [9,4] 도착 또는 max_moves (=350) 초과하면 종료
        j += 1
        mov += 1
        # 상태 state1에서 action 결정
        #####################################################################################
        if env.curloc == [9,4]:  # 출발점에서는 무조건 위로 올라간다 (action=0)
            action = 0
        elif env.curloc[0] == 0 and env.curloc[1] in [0,1,2,3,4,5,6,7,8]:  # (0,0)~(0,8)에서는 무조건 아래로
            action = 1
        elif env.curloc[0] in [2,3,4,5] and env.curloc[1] == 8:  # (2,8)~(5,8)에서는 무조건 왼쪽으로
            action = 2
        elif env.curloc[0] in [2,3,4,5] and env.curloc[1] == 0:  # (2,0)~(5,0)에서는 무조건 오른쪽으로
            action = 3
        else:  # 위 경우를 제외하곤 e-greedy로 액션 선택
            action = q.sample_action(history1, epsilon)
        #####################################################################################
        # action = q.sample_action(state1, epsilon)  # 위 조건을 적용 안 할 경우
        # make Move... action에 따른 이동
        gridmap, reward, cum_reward, done = env.step(action)
            # self.actions.append((new_x, new_y))...새로 도착한 위치를 경로 리스트에 추가
            # 누적 보상 계산
        ###########################
        # print(gridmap.reshape(10,9))
        ###########################

        #####################################################################################
        # next input 만들어주기
        next_gridmap = np.reshape([gridmap],(1,1,10,9))                                             # 다음 그리드를 히스토리 사이즈에 맞게 reshape
        next_gridmap = torch.from_numpy(next_gridmap).float()                                       # 텐서로 변환
        history2 = torch.cat([next_gridmap,history1[:,:3,:,:]],dim=1) # (1,1+3,10,9)                # 지금 있는 히스토리중 마지막 채널을 지우고 new를 제일 앞에 붙임

        done_mask = 0.0 if done else 1.0
        memory.put((history1, action, reward, history2, done_mask))  # 경험을 메모리에 저장
        history1 = history2
        #####################################################################################

        # if memory.size() > train_start:  # 메모리에 train_start 크기 이상 쌓이면... 미니배치 훈련
        if j > train_start:  # 메모리에 미니배치 크기 이상 쌓이면... 미니배치 훈련
            #print('memory size:', memory.size())
            loss = train(q, q_target, memory, optimizer)
            if j % print_interval == 0:
                print('epiode #:', n_epi, 'loss:', loss, 'j:', j)
            losses.append(loss)
            cum_rewards.append(cum_reward)
            moves.append(mov)
            

            if j % sync_freq == 0:  # sync_freq마다 q 네트워크 파라미터를 q_target 네트워크로 복사
                q_target.load_state_dict(q.state_dict())

        ############## 현황 모니터링 #############################################################
        if done and j % 100 == 0:  # 100번 마다 mov 횟수 저장:
        #     moves.append(mov)
            print('종료: done =', done, '... j =', j, ' move =', mov)
        if env.goal_ob_reward:  # 최종 목적지 도달한 경우에만 화면에 표시
            goal_ob_reward_count += 1
            print(env.local_target_original, '종료: env.goal_ob_reward =', env.goal_ob_reward, '... j =', j, ' move =', mov, '@ 에피소드 #', n_epi)
            print(f"{n_epi}번째 에피소드까지 총 {goal_ob_reward_count}번 finish 했습니다.")
            ### finish record 저장하는 txt파일 만들기
            with open(__file__ + '/eps_log.txt', 'a') as finish_record:
                finish_record.write(str(goal_ob_reward_count)+" "+str(env.local_target_original[-2])+" "+str(cum_reward)+" "+\
                                    str(mov)+" "+str(j)+" "+str(n_epi)+"\n")
        ##########################################################################################

        if done or mov > max_moves:
            mov = 0
            done = True
    
    # while loop 종료 ----- 최종 목적지 [9,4] 도착 또는 max_moves (=350) 초과

    if j > train_start:  # 메모리에 train_start 크기 이상 쌓이면... 미니배치 훈련
        if (n_epi % 10000 == 0 and n_epi != 0) or n_epi >= epochs - 1:
            torch.save({'epoch': n_epi, 
                        'model_state_dict': q.state_dict(), 
                        'optimizer_state_dict': optimizer.state_dict(), 
                        'loss': loss, 
                        }, PATH)
            print('▶ 모델 저장됨!!! @ 에피소드', n_epi)

        ##### gif 생성을 위한 경로(env.actions) 저장 ####################################    
        if n_epi % 1000 == 0 or env.goal_ob_reward:
            f.write(str(n_epi)+'/'+str(env.local_target_original)+'\n')
            f.write(str(env.actions))
            f.write('\n')
        ######################################### 
f.close()
#########################################
## 프로그램 시작 및 종료 시간 출력
cur_time = datetime.now(tz)
end_time = cur_time.strftime("%H:%M:%S")
print ('실행 종료!', 'Start@', start_time, 'End@', end_time)
        #cur_time = datetime.now(tz)
        #simple_cur_time = cur_time.strftime("%H:%M:%S")
        #print('▶ Episode #', iter, 'start time:', simple_cur_time, end='→')

print(gridmap.reshape(10,9))  # 최종 그리드맵 확인

## loss 그래프
plt.figure(figsize=(6,4))
plt.plot(losses)
plt.xlabel("# of Actions (j)",fontsize=12)
plt.ylabel("Loss",fontsize=12)

## 경로길이(이동 횟수) 그래프
plt.figure(figsize=(6,4))
plt.plot(moves)
plt.xlabel("# of Actions (j)",fontsize=12)
plt.ylabel("Moves",fontsize=12)

## 보상값(cum_rewards) 그래프
plt.figure(figsize=(6,4))
plt.plot(cum_rewards)
plt.xlabel("# of Actions (j)",fontsize=12)
plt.ylabel("Cumulative Rewards",fontsize=12)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[[4, 0], [2, 0], [0, 1], [0, 2], [0, 3], [4, 8], [5, 8], [9, 4]] 종료: env.goal_ob_reward = True ... j = 717313  move = 42 @ 에피소드 # 27165
27165번째 에피소드까지 총 7972번 finish 했습니다.
[[5, 0], [0, 8], [2, 8], [3, 8], [4, 8], [5, 8], [9, 4]] 종료: env.goal_ob_reward = True ... j = 717399  move = 45 @ 에피소드 # 27167
27167번째 에피소드까지 총 7973번 finish 했습니다.
[[2, 0], [0, 3], [0, 4], [0, 5], [0, 7], [3, 8], [5, 8], [9, 4]] 종료: env.goal_ob_reward = True ... j = 717448  move = 49 @ 에피소드 # 27168
27168번째 에피소드까지 총 7974번 finish 했습니다.
[[5, 0], [3, 0], [0, 2], [0, 4], [5, 8], [9, 4]] 종료: env.goal_ob_reward = True ... j = 717486  move = 38 @ 에피소드 # 27169
27169번째 에피소드까지 총 7975번 finish 했습니다.
[[4, 0], [3, 0], [0, 0], [0, 4], [0, 5], [0, 6], [0, 7], [9, 4]] 종료: env.goal_ob_reward = True ... j = 717530  move = 44 @ 에피소드 # 27170
27170번째 에피소드까지 총 7976번 finish 했습니다.
[[0, 0], [0, 2], [0, 3], [0, 6], [3, 8], [4, 8], [9, 4]] 종료: env.goal_ob_reward = True ... j = 717574  move = 44 @ 에피소드 # 27171


## 테스트 코드

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # GPU를 사용할 경우 ???
print('tensor in', device)  # GPU 사용 확인
PATH = __file__ + '/eps.pt' ## PATH 경로 쉽게 수정
train_mode = False
## test 코드 #################################################################################
def test(epochs=1000, train_mode=False, display=False, model_load=True, PATH=PATH):  # train_mode = False 테스트 모드

    test_env = Simulator()  # 데이터셋 읽기. 그리드월드 크기 지정
    test_q = Qnet()
    test_q.to(device)
    learning_rate = 0.00025    #3. 0.0005
    optimizer = optim.Adam(test_q.parameters(), lr=learning_rate)

    if model_load:
        ###############################################################
        ## 학습한 모델 불러오기
        ###############################################################
        checkpoint = torch.load(PATH)
        # checkpoint = torch.load(PATH, map_location=torch.device('cpu'))  ###### gpu 저장 모델 cpu 로드시 #####
        test_q.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        test_q.eval()

    epochs = len(test_env.files)
    wins = 0
    j = 0
    total_reward = 0.0

    ##### 이미 존재하는 테스트 결과 저장한 파일 지우기
    test_record_file_path = __file__ + '/eps_test_record.txt'
    if os.path.exists(test_record_file_path):
        os.remove(test_record_file_path)
    ##### 이미 존재하는  gif 생성을 위한 파일 지우기 ####################################    
    test_gif_file_path = __file__ + '/eps_test_gif.txt'
    if os.path.exists(test_gif_file_path):
        os.remove(test_gif_file_path)

    for n_epi in range(epochs):
        gridmap = test_env.reset(n_epi)  # 에피소드 시작시 12개 값 초기화
        history_ = np.stack((gridmap,gridmap,gridmap,gridmap), axis = 0) # 초기 state로 히스토리 초기화 (4,10,9)
        history = np.reshape([history_], (1,4,10,9)) # (배치, 채널, 가로, 세로) 로 reshape
        history = torch.from_numpy(history).float() # 텐서로 변환   
        done = False
        mov = 0
        while (not done):  # 최종 목적지 [9,4] 도착
            mov += 1
            j += 1
            # 상태 state1에서 action 결정
            #####################################################################################
            if test_env.curloc == [9,4]:  # 출발점에서는 무조건 위로 (action=0)
                action = 0
            elif test_env.curloc[0] == 0 and test_env.curloc[1] in [0,1,2,3,4,5,6,7,8]:  # (0,0)~(0,8)에서는 무조건 아래로
                action = 1
            elif test_env.curloc[0] in [2,3,4,5] and test_env.curloc[1] == 8:  # (2,8)~(5,8)에서는 무조건 왼쪽으로
                action = 2
            elif test_env.curloc[0] in [2,3,4,5] and test_env.curloc[1] == 0:  # (2,0)~(5,0)에서는 무조건 오른쪽으로
                action = 3
            else:  # 위 경우를 제외하곤 test 액션 선택
                action = test_q.test_action(history)
            #####################################################################################
            # action = test_q.test_action(history)  # 위 조건을 적용하지 않을 경우
            # make Move... action에 따른 이동
            gridmap, reward, cum_reward, done = test_env.step(action)

            next_gridmap_ = np.reshape([gridmap],(1,1,10,9)) # 다음 그리드를 히스토리 사이즈에 맞게 reshape
            next_gridmap = torch.from_numpy(next_gridmap_).float() # 텐서로 변환
            new_history = torch.cat([next_gridmap, history[:,:3,:,:]],dim=1) # (1,1+3,10,9)
            # 지금 있는 히스토리중 마지막 채널을 지우고 new를 제일 앞에 붙임
            history = new_history

            if display:
                print('Move #%s; Taking action: %s' % (mov, action))
                print(gridmap.reshape(10,9))

            if test_env.goal_ob_reward:
                wins += 1
                if display:
                    print("Game won! Reward: %s" % (cum_reward))
                    
            if (mov > 50):  # 경로 길이가 너무 길어서 제외시키는 조건 추가 필요시 사용
                if display:
                    print("Game lost; too many moves.")
                break
        
        total_reward += cum_reward
        avg_reward = total_reward / (n_epi+1)

        print('Episode: {0:04d}   Timestep: {1:05d}   Moves: {2:02d}   Average Reward: {3:0.2f}   Finish Rate: {4:0.2f}'.\
              format(n_epi, j, mov, avg_reward, wins/(n_epi+1)))
        ### 테스트 결과 저장하는 txt파일 만들기
        with open(test_record_file_path, 'a') as test_record:
            test_record.write("Episode: "+str(n_epi)+" Timestep:"+str(j)+" Moves: "+str(mov)+" Average Reward: "+\
                                str(avg_reward)+" Finish Rate: "+str(wins/(n_epi+1))+"\n")

        ##### gif 생성을 위한 경로(env.actions) 저장 ####################################    
        with open(test_gif_file_path, 'a') as test_gif:
            test_gif.write(str(n_epi)+'/'+str(test_env.local_target_original)+'\n')
            test_gif.write(str(test_env.actions))
            test_gif.write('\n')
        ######################################### 

    win_perc = float(wins) / float(epochs)
    print("Test performed: {0}, # of wins: {1}".format(epochs,wins))
    print("Win percentage: {}%".format(100.0*win_perc))
    return win_perc

tensor in cuda


In [13]:
test(display=False)

data/factory_order_test.csv used
Episode: 0000   Timestep: 00044   Moves: 44   Average Reward: 81.94   Finish Rate: 1.00
Episode: 0001   Timestep: 00086   Moves: 42   Average Reward: 81.82   Finish Rate: 1.00
Episode: 0002   Timestep: 00126   Moves: 40   Average Reward: 81.74   Finish Rate: 1.00
Episode: 0003   Timestep: 00170   Moves: 44   Average Reward: 76.53   Finish Rate: 1.00
Episode: 0004   Timestep: 00210   Moves: 40   Average Reward: 75.77   Finish Rate: 1.00
Episode: 0005   Timestep: 00252   Moves: 42   Average Reward: 76.95   Finish Rate: 1.00
Episode: 0006   Timestep: 00294   Moves: 42   Average Reward: 76.23   Finish Rate: 1.00
Episode: 0007   Timestep: 00336   Moves: 42   Average Reward: 77.05   Finish Rate: 1.00
Episode: 0008   Timestep: 00378   Moves: 42   Average Reward: 76.36   Finish Rate: 1.00
Episode: 0009   Timestep: 00420   Moves: 42   Average Reward: 77.01   Finish Rate: 1.00
Episode: 0010   Timestep: 00464   Moves: 44   Average Reward: 77.46   Finish Rate: 1.00

1.0